In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import os
chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver


driver = webdriver.Chrome(chromedriver)

In [2]:
#get movie titles to run through
movies = pd.read_csv('movie_metadata.csv')
titles = movies.movie_title

In [4]:
a_scores = {}
c_scores = {}
errors = 0
missed_titles = []

In [5]:
#run this until timeout basically
url = "https://www.google.com/"
driver.get(url)
driver.implicitly_wait(3)

for title in titles :
    try :
        #search title on google and click the rotten tomatoes link if avaiable
        title=title.strip()
        title_p = title+' movie'
        query = driver.find_element_by_name("q")
        query.send_keys(title_p)
        query.send_keys(Keys.RETURN)
        poss_links = driver.find_elements_by_class_name('NY3LVe')
        for link in poss_links :
            if 'Rotten Tomatoes' in link.text :
                link.click()
                break
        #get audience and critic score from rotten tomatoes
        c_score = driver.find_element_by_class_name('mop-ratings-wrap__percentage').text.strip('%')
        a_score = driver.find_elements_by_xpath("//*[contains(@class,'mop-ratings-wrap__percentage mop-ratings-wrap__percentage--audience')]")[0].text.split('%')[0]
        c_scores[title] = c_score
        a_scores[title] = a_score

        driver.back()
        
    except :
        #either a missing link or a missing score
        errors += 1
        missed_titles.append(title)
        
    #this makes sure that we don't get stuck on rotten tomatoes when the page is missing a score
    if 'google' not in driver.current_url :
        driver.back()
    #clear the search!
    driver.find_element_by_name("q").clear()



TimeoutException: Message: timeout
  (Session info: chrome=73.0.3683.103)
  (Driver info: chromedriver=2.35.528157 (4429ca2590d6988c0745c24c8858745aaaec01ef),platform=Mac OS X 10.14.3 x86_64)


In [7]:
len(a_scores)

2328

### SAVE!
Before you have to wait all that time again.

In [43]:
final_part_1 = pd.DataFrame([
    a_scores,
    c_scores
    ])
final_part_1 = final_part_1.transpose()

In [44]:
final_part_1.to_pickle('./ratings1.pkl')

In [25]:
#timeout happened for me at The Salton Sea (can find approximate location by errors + scores); 
#you can see where also by looking at your chrome window
titles_2 = titles[2463:]

In [33]:
#same code as above run since timeout
driver = webdriver.Chrome(chromedriver)

url = "https://www.google.com/"
driver.get(url)
driver.implicitly_wait(3)

a_scores2 = {}
c_scores2 = {}

for title in titles_2 :
    try :
        title=title.strip()
        title_p = title+' movie'
        query = driver.find_element_by_name("q")
        query.send_keys(title_p)
        query.send_keys(Keys.RETURN)
        poss_links = driver.find_elements_by_class_name('NY3LVe')
        for link in poss_links :
            if 'Rotten Tomatoes' in link.text :
                link.click()
                break

        c_score = driver.find_element_by_class_name('mop-ratings-wrap__percentage').text.strip('%')
        a_score = driver.find_elements_by_xpath("//*[contains(@class,'mop-ratings-wrap__percentage mop-ratings-wrap__percentage--audience')]")[0].text.split('%')[0]
        c_scores2[title] = c_score
        a_scores2[title] = a_score

        driver.back()
    except :
        errors += 1
        missed_titles.append(title)
    if 'google' not in driver.current_url :
        driver.back()
    driver.find_element_by_name("q").clear()

In [45]:
final_part_2 = pd.DataFrame([
    a_scores2,
    c_scores2
    ])
final_part_2 = final_part_2.transpose()

In [46]:
final_part_2.to_pickle('./ratings2.pkl')

In [80]:
final = pd.concat([final_part_1,final_part_2],axis=0).reset_index()

In [82]:
final.columns = ['movie_title','audience','critic']
final.to_pickle('./ratings.pkl')

In [83]:
final.to_csv('./ratings.csv')

# This was my initial attempt at webscraping
But it only returned 546 values (out of **5043**), and some of those values were messed up, so I had to *pivot*.

In [ ]:
#import re
#import time



# url = "https://www.rottentomatoes.com/"
# driver.get(url)
# driver.implicitly_wait(3)

# errors = 0
# critic_scores = {}
# audience_scores = {}
# for title in titles :
#     try :
#         title=title.strip()
#         clean_title = re.findall('(\w|\s|-)',title)
#         #if title == "Pirates of the Caribbean: At World's End" :
#         #    title = 'Pirates of the Caribbean: At Worlds End'
#         query = driver.find_element_by_name("search")
#         query.send_keys(clean_title)
#         query.send_keys(Keys.RETURN)
#         driver.find_element_by_link_text(title).click()
#         c_score = driver.find_element_by_class_name('mop-ratings-wrap__percentage').text.strip('%')
#         a_score = driver.find_elements_by_xpath("//*[contains(@class,'mop-ratings-wrap__percentage mop-ratings-wrap__percentage--audience')]")[0].text.split('%')[0]
#         critic_scores[title] = c_score
#         audience_scores[title] = a_score
#     except :
#         errors += 1
#         continue
#     driver.find_element_by_name("search").clear()
#     time.sleep(.2)
# driver.quit()

# final_frame = pd.DataFrame([critic_scores,audience_scores])
# final_frame = final_frame.transpose()
#final_frame.to_pickle('./ratings.pkl')